In [1]:
import pandas as pd
import os
import pyarrow
import openpyxl


In [ ]:
x_train = pd.read_parquet(os.path.join('..', 'Датасеты', 'X_train.parquet'), engine='pyarrow')
y_train = pd.read_parquet(os.path.join('..', 'Датасеты', 'y_train.parquet'), engine='pyarrow')

In [ ]:

from exgauster.utils import resample_dataset as resample_dataset, fillna_dataset_ffill as fillna_dataset


In [ ]:
x_train_unit4_all = fillna_dataset(x_train[[x for x in x_train.columns if 'ЭКСГАУСТЕР 4' in x ]])
y_train_unit4_all = fillna_dataset(y_train[[x for x in y_train.columns if 'ЭКСГАУСТЕР А/М №4' in x]])

# resample_rule = '12H'
resample_rule = '5min'

x_train_4_resampled = resample_dataset(x_train_unit4_all, resample_rule)


In [ ]:
def is_m3(cell):
    return cell == 2

def is_m1(cell):
    return cell == 1


In [ ]:
y_train_unit4_all_m1_mask = y_train_unit4_all.applymap(is_m1)
y_train_unit4_all_m3_mask = y_train_unit4_all.applymap(is_m3)

y_train_unit4_all_m1 = y_train_unit4_all.copy()
y_train_unit4_all_m3 = y_train_unit4_all.copy()

y_train_unit4_all_m1[y_train_unit4_all_m3_mask] = 0
y_train_unit4_all_m1[y_train_unit4_all_m1_mask] = 1
y_train_unit4_all_m3[y_train_unit4_all_m3_mask] = 1
y_train_unit4_all_m3[y_train_unit4_all_m1_mask] = 0



In [ ]:
y_train_4_m1_resampled = resample_dataset(y_train_unit4_all_m1, resample_rule)
y_train_4_m3_resampled = resample_dataset(y_train_unit4_all_m3, resample_rule)


target_variables_4_pd_m1 = []
for colname in y_train_4_m1_resampled.columns:
    target_variables_4_pd_m1.append(y_train_4_m1_resampled[[colname]])


target_variables_4_pd_m3 = []
for colname in y_train_4_m3_resampled.columns:
    target_variables_4_pd_m3.append(y_train_4_m3_resampled[[colname]])










In [ ]:

x_train_unit4 = x_train_4_resampled
x_train_unit_4_0 = x_train_unit4
# x_train_unit_4_0 = pd.merge(x_train_unit4, target_variables_4_pd_m1[0], left_index=True, right_index=True, how='outer')
x_train_unit_4_0 = pd.merge(x_train_unit_4_0, target_variables_4_pd_m3[0], left_index=True, right_index=True, how='outer')

x_train_unit_4_0.to_parquet(os.path.join('..', 'x_train_unit_4_0.parquet'))




In [ ]:
len(x_train_4_resampled)

In [ ]:
messages=pd.read_excel(os.path.join('..', 'Датасеты', 'messages.xlsx'))
test_intervals=pd.read_excel(os.path.join('..', 'Датасеты', 'test_intervals.xlsx'))

In [ ]:
x_train_unit4 = x_train_4_resampled


In [ ]:
for i, name in enumerate(x_train_unit4.columns):
    print(i, name)

In [ ]:
type(x_train_unit4)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from merlion.utils.time_series import TimeSeries

In [ ]:
len(target_variables_4_pd_m1)

In [ ]:
# x_train_unit_4_0_sorted = x_train_unit_4_0.sort_values(by=x_train_unit_4_0.index)
x_train_unit_4_0_sorted = x_train_unit_4_0

In [ ]:
split_index = int(len(x_train_unit_4_0_sorted) * 0.8)

x_unit_4_0_train = x_train_unit_4_0_sorted.iloc[:split_index]
x_unit_4_0_test = x_train_unit_4_0_sorted.iloc[split_index:]


In [ ]:
len(x_unit_4_0_test)

In [ ]:
x_unit_4_0_train = x_unit_4_0_train.apply(pd.to_numeric)
x_unit_4_0_test = x_unit_4_0_test.apply(pd.to_numeric)

In [ ]:
m_x_train_4_0 = TimeSeries.from_pd(x_unit_4_0_train)
m_x_train_4_0_aligned = m_x_train_4_0.align()


m_x_test_4_0 = TimeSeries.from_pd(x_unit_4_0_test)
m_x_test_4_0_aligned = m_x_test_4_0.align()

In [ ]:
m_x_train_4_0_aligned[0][1][16]

In [ ]:
len(m_x_train_4_0_aligned)

In [ ]:
len(x_unit_4_0_test)

In [ ]:
if 0:
    for model in [model1, model2, model3]:
        print(f"Training {type(model).__name__}...")
        train_pred, train_stderr = model.train(m_x_train_4_0_aligned)
        train_results.append([train_pred, train_stderr])

from merlion.evaluate.forecast import ForecastMetric
from merlion.models.factory import ModelFactory
from merlion.models.ensemble.combine import ModelSelector, Mean


# Time series is sampled hourly, so max_forecast_steps = 24 means we can predict up to 1 day in the future
# target_seq_index = target_index
target_seq_index = 16
max_forecast_steps = len(x_unit_4_0_test)
print(max_forecast_steps)
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

if 0:
    model1 = ModelFactory.create("DefaultForecaster", **kwargs)
    model2 = ModelFactory.create("Arima", **kwargs)

    # This ModelSelector combiner picks the best model based on sMAPE
    model3 = ModelFactory.create("ForecasterEnsemble", models=[model1, model2],
                                 combiner=ModelSelector(metric=ForecastMetric.Mean))




In [ ]:
target_seq_index = 16
max_forecast_steps = 100

In [ ]:
from merlion.models.forecast.trees import LGBMForecasterConfig

import lgbm_forecaster_custom
from importlib import reload
import timeit

reload(lgbm_forecaster_custom)

window_size = 60
lgbm_config = LGBMForecasterConfig(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps, maxlags=window_size)
model_lgbm = lgbm_forecaster_custom.LGBMForecasterCustom(lgbm_config, window_size=window_size)


train_results = []
print(f"Training {type(model_lgbm).__name__}...")


def learn():
    shuffle = TimeSeries.from_pd(m_x_train_4_0_aligned.to_pd().sample(frac=1))
    train_pred, train_stderr = model_lgbm.train(shuffle)
    train_results.append([train_pred, train_stderr])

learn_time = timeit.timeit(learn, number=1)
print('train time', learn_time / 60, 'min')


from merlion.evaluate.forecast import ForecastMetric

target_univariate = m_x_test_4_0_aligned.univariates[m_x_test_4_0_aligned.names[target_seq_index]]
target = target_univariate[:max_forecast_steps].to_ts()
target = target_univariate.to_ts()
# print(target)

models_list = [model_lgbm]
# models_list = [model1, model2, model3]
forecast = None
stderr = None

for model in models_list:
    # forecast, stderr = model.forecast(target.time_stamps)
    def forecast():
        global forecast
        global stderr

        forecast, stderr = model.forecast(m_x_train_4_0_aligned, m_x_test_4_0_aligned)
        rmse = ForecastMetric.RMSE.value(ground_truth=target, predict=forecast)
        smape = ForecastMetric.sMAPE.value(ground_truth=target, predict=forecast)
        print(f"{type(model).__name__}")
        print(f"RMSE:  {rmse:.4f}")
        print(f"sMAPE: {smape:.4f}")
        print('window_size: ', window_size)
    learn_time = timeit.timeit(forecast, number=1)
    print('forecast_time', learn_time, 'sec')


import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

model_index = 0
model_plot = models_list
model = model_plot[model_index]
train_forecast = train_results[model_index][0]

# создаем объект figure
fig = plt.figure(figsize=(20, 10))
# добавляем объект axes на figure
ax = fig.add_subplot(111)

target_name_col = 'Y_ЭКСГАУСТЕР А/М №4_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №4'
ax.plot(forecast.index, forecast[target_name_col], label='Прогноз')

# fig, ax = model.plot_forecast(time_stamps=target.time_stamps, plot_forecast_uncertainty=True, time_series_prev=m_x_train_4_0_aligned)

ax.plot(target.index, target.univariates[target_name_col], label='Реальное значение', color="#303030")
ax.plot(x_unit_4_0_train.index, x_unit_4_0_train[target_name_col], color="#303030")
ax.plot(train_forecast.index, train_forecast.univariates[target_name_col], label='Прогноз при обучении', color="red")

ax.set_xlim(x_unit_4_0_train.index[0], target.index[-1])
ax.legend()
plt.show()

In [ ]:
len_of_indices = []
len_of_lags = []
len_of_names = []
max_indices = len(model_lgbm.model.feature_importances_)
tmplt = list(range(16))
for i in range(max_indices // 16 + 1):
    len_of_indices.extend(tmplt)
    len_of_lags.extend([i for x in range(16)])
    len_of_names.extend(list(x_train_unit4.columns))

len_of_indices = len_of_indices[:max_indices]
len_of_lags = len_of_lags[:max_indices]
len_of_names = len_of_names[:max_indices]
importances = pd.DataFrame({
    'importances': model_lgbm.model.feature_importances_,
    'indices': list(range(len(model_lgbm.model.feature_importances_))),
    'feature_number': len_of_indices,
    'feature_lag': len_of_lags,
    'names': len_of_names
}).sort_values(by=['importances', 'feature_number', 'feature_lag'], ascending=[False, True, True])


from IPython.display import display


with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(importances)
